In [70]:
# Imports
from pickle import dump, load
import nltk
from nltk import word_tokenize,FreqDist
import re
from nltk.corpus import wordnet as wn

In [2]:
doc_list =load(open("data/Proxima_doc_content",'rb'))
len(doc_list)

4

In [4]:
# Start working on one document and associated comments
document = doc_list[0]
document.keys()

dict_keys(['comment_list', 'text'])

In [5]:
# Convert bs4 ResultSet to string
document_text = []
for item in document['text']:
    document_text.append(str(item))

In [6]:
text = document_text[0].replace('\n',' ')
text

'<pre> [Federal Register Volume 79, Number 198 (Tuesday, October 14, 2014)] [Proposed Rules] [Pages 61587-61590] From the Federal Register Online via the Government Printing Office [<a href="http://www.gpo.gov">www.gpo.gov</a>] [FR Doc No: 2014-24324]    ========================================================================  Proposed Rules                                                  Federal Register  ________________________________________________________________________    This section of the FEDERAL REGISTER contains notices to the public of   the proposed issuance of rules and regulations. The purpose of these   notices is to give interested persons an opportunity to participate in   the rule making prior to the adoption of the final rules.    ========================================================================      Federal Register / Vol. 79, No. 198 / Tuesday, October 14, 2014 /  Proposed Rules    [[Page 61587]]    DEPARTMENT OF THE INTERIOR  National Park Service  36 

In [7]:
summary_idx = text.find("SUMMARY:")
dates_idx = text.find("DATES:")
supp_info_idx = text.find("SUPPLEMENTARY INFORMATION:")
summary = text[summary_idx:dates_idx]
suppl_info = text[supp_info_idx:]

In [8]:
# Get rid of HTML tags
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [9]:
clean_suppl_info = cleanhtml(suppl_info)

In [64]:
clean_suppl_info

"SUPPLEMENTARY INFORMATION:  Background  Legislation and Purposes of Cuyahoga Valley National Park      On December 27, 1974, President Gerald Ford signed Public Law 93- 555 creating Cuyahoga Valley National Recreation Area for the purpose  of ``preserving and protecting for public use and enjoyment, the  historic, scenic, natural, and recreational values of the Cuyahoga  River and the adjacent lands of the Cuyahoga Valley and for the purpose  of providing for the maintenance of needed recreational open space  necessary to the urban environment.'' In 2000, Congress redesignated  Cuyahoga Valley National Recreation Area as Cuyahoga Valley National  Park (CVNP or Park) with the passage of the Department of the Interior  and Related Agencies Appropriations Act (Pub. L. 106-291).     Cuyahoga Valley National Park is an important national resource  within a predominantly metropolitan region, where the Park is visited  by approximately 2,500,000 people annually. Located in Cuyahoga and  Summ

** Now extract keywords from supplement information **

In [59]:
def tokenize_text(corpus):
    pattern = r'''(?x)    # set flag to allow verbose regexps
    (([A-Z]\.)+)       # abbreviations, e.g. B.C.
    |(\w+([-']\w+)*)       # words with optional internal hyphens e.g. after-ages or author's
    '''
    tokens = nltk.regexp_tokenize(corpus,pattern)
    all_token = [word.lower() for token in tokens for word in token if word != "" and word[0] != "'" and word[0] != "-"]
    return all_token

def tokenize_text_sent(corpus):
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus) # Split text into sentences    
    return [tokenize_text(sent) for sent in raw_sents]

def tag_my_text(sents):
    return [nltk.pos_tag(sent) for sent in sents]

In [60]:
suppl_info_tokens = tokenize_text(clean_suppl_info)
suppl_info_sent_tokens = tokenize_text_sent(clean_suppl_info)
pos_tag_text = tag_my_text(suppl_info_sent_tokens)

In [37]:
# matchobj = re.findall(r'(January|February|March|April|May|June|July|August|September|October|November|December)
#(\s\d{2}\,\s\d{4})',clean_suppl_info)

144 sentences

In [51]:
#Chunk noun phrases in tree 
def noun_phrase_chunker():
    grammar = r"""
    NP: {<DT|PP\$>?<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
    """
    cp = nltk.RegexpParser(grammar)
    return cp

#Extract only the NP marked phrases from the parse tree, that is the chunk we defined
def noun_phrase_extractor(sentences, chunker):
    res = []
    for sent in sentences:
        tree = chunker.parse(sent)
        for subtree in tree.subtrees():
            if subtree.label() == 'NP' : 
                res.append(subtree[0:len(subtree)])
                #res.append(subtree[0])
                #print(subtree)
    return res

In [52]:
#remove tags and get only the noun phrases , can be adjusted for length
def noun_phrase_finder(tagged_text):
    all_proper_noun = noun_phrase_extractor(tagged_text,noun_phrase_chunker()) 
    #does not literally mean proper noun. Chunker only extracts common noun
    noun_phrase_list = []                                                      
    #noun_phrase_string_list =[]
    for noun_phrase in all_proper_noun:
        if len(noun_phrase) > 0: #this means where the size of the phrase is greater than 1
            small_list =[]
            for (word,tag) in noun_phrase:
                small_list.append(word)
            noun_phrase_list.append(small_list)
            #noun_phrase_string_list.append(' '.join(small_list))
    return noun_phrase_list

In [61]:
np_list = noun_phrase_finder(pos_tag_text)

In [55]:
#get frequency dist of different length in all the noun phrases extracted. 
#Something of the form {1:45,2:23} - how many 1phrased and 2 phrased chunks I have etc.
def get_length_np(nounPhrase):
    np_length={}
    for inner_np in nounPhrase:
        np_length[len(inner_np)] = np_length.get(len(inner_np),0) + 1
    return np_length

#get freq dist obj for noun phrase of different lengths
def find_freq(nested_list,nest_len):
    #from nltk.probability import FreqDist
    fdist_list =[]
    for inner_np in nested_list:
        if len(inner_np) == nest_len:
            fdist_list.append(' '.join(inner_np))
    fdist = FreqDist(fdist_list)
    return fdist

#make a grand list of top occuring noun phrases of different sizes
def get_top_np(np):
    master_common_list=[]
    len_list =get_length_np(np).keys()
    for item in len_list:
        fdist_np = find_freq(np_list,item)
        master_common_list.append(fdist_np.most_common(15))
    return master_common_list

In [63]:
grand_list= get_top_np(np_list)
grand_list

[[('trail', 35),
  ('plan', 22),
  ('use', 19),
  ('bicycle', 17),
  ('management', 12),
  ('cuyahoga', 11),
  ('order', 10),
  ('executive', 10),
  ('information', 9),
  ('connector', 7),
  ('program', 7),
  ('b', 6),
  ('park', 6),
  ('valley', 6),
  ('cvnp', 6)],
 [('the park', 29),
  ('this rule', 16),
  ('national park', 9),
  ('the superintendent', 9),
  ("the park's", 7),
  ('the cuyahoga', 6),
  ('u.s.c. c.', 5),
  ('the office', 4),
  ('the trail', 4),
  ('public notice', 3),
  ('old carriage', 3),
  ('the nps', 3),
  ('public use', 3),
  ('executive order', 3),
  ('nps gov', 3)],
 [('the trails forever', 2),
  ('the private sector', 2),
  ('the old carriage', 2),
  ('the national park', 2),
  ("area's natural scenic", 2),
  ('old akron-peninsula road', 1),
  ('a critical component', 1),
  ('a special regulation', 1),
  ('the joint volunteer', 1),
  ('the first time', 1),
  ('the north side', 1),
  ('the presidential memorandum', 1),
  ('an open exchange', 1),
  ('valley natio

In [65]:
# Lesk algorith for disambiguation in case of multiple synsets of a word
def compare_overlaps_greedy(context, synsets_signatures, pos=None):
    """
    Calculate overlaps between the context sentence and the synset_signature
    and returns the synset with the highest overlap.
    
    :param context: ``context_sentence`` The context sentence where the ambiguous word occurs.
    :param synsets_signatures: ``dictionary`` A list of words that 'signifies' the ambiguous word.
    :param pos: ``pos`` A specified Part-of-Speech (POS).
    :return: ``lesk_sense`` The Synset() object with the highest signature overlaps.
    """
    # if this returns none that means that there is no overlap
    max_overlaps = 0
    lesk_sense = None
    for ss in synsets_signatures:
        if pos and str(ss.pos()) != pos: # Skips different POS.
            continue
        overlaps = set(synsets_signatures[ss]).intersection(context)
        if len(overlaps) > max_overlaps:
            lesk_sense = ss
            max_overlaps = len(overlaps)  
    return lesk_sense

def lesk(context_sentence, ambiguous_word, pos=None, dictionary=None):
    """
    This function is the implementation of the original Lesk algorithm (1986).
    It requires a dictionary which contains the definition of the different
    sense of each word. See http://goo.gl/8TB15w

        >>> from nltk import word_tokenize
        >>> sent = word_tokenize("I went to the bank to deposit money.")
        >>> word = "bank"
        >>> pos = "n"
        >>> lesk(sent, word, pos)
        Synset('bank.n.07')
    
    :param context_sentence: The context sentence where the ambiguous word occurs.
    :param ambiguous_word: The ambiguous word that requires WSD.
    :param pos: A specified Part-of-Speech (POS).
    :param dictionary: A list of words that 'signifies' the ambiguous word.
    :return: ``lesk_sense`` The Synset() object with the highest signature overlaps.
    """
    if not dictionary:
        dictionary = {}
        for ss in wn.synsets(ambiguous_word):
            dictionary[ss] = ss.definition().split()
    best_sense = compare_overlaps_greedy(context_sentence, dictionary, pos)
    return best_sense
    #return dictionary 



In [72]:
# this function takes in a word and gets the most relevant synset based on context from the text. 
# for exact algorith refer the text above ("what I want to do" markdown)
def get_synset(word, pos):
    if len(wn.synsets(word)) == 1:
        #print("here1")
        return wn.synsets(word)[0]
    else:
        #get all context sentences
        all_sent =[]
        for sent in pos_tag_text:
            for (w,t) in sent:
                if w == word:
                    all_sent.append(sent)
        #call lesk here
        app_syn = lesk(all_sent[len(all_sent)//2], word, pos)
        if app_syn != None:
            #print("here2")
            return app_syn
        else:
            #second lesk trial with another context sentence
            app_syn = lesk(all_sent[len(all_sent)//3], word, pos)
            if app_syn != None:
                #print("here2")
                return app_syn
            else:
                #give up and choose 1st synset from list with matching pos
                #print("here3")
                all_syns = wn.synsets(word)
                for syn in all_syns:
                    #print(syn.pos())
                    if syn.pos() == pos:
                        return syn
    return False

In [67]:
# this functions take all the single and double legth phrases form grand_list and gets sysnset for all them. (1 each)
def get_singles_doubles_synset():
    single_double_synset =[]
    #get synsets of all singletons
    for (singles,count) in grand_list[0]:
        singles_syn = get_synset(singles, 'n')
        if singles_syn:
            single_double_synset.append(singles_syn)    
    #get sysnset of all doubles
    for (doubles,count) in grand_list[1]:
        doubles_syn = get_synset(doubles.split()[1], 'n')
        if doubles_syn:
            single_double_synset.append(doubles_syn)
    return single_double_synset

In [68]:
# Direct concepts
def get_direct_concept():
    #get all relevant sysnsets
    all_synsets = get_singles_doubles_synset()
    #extraxting concepts
    lemma_names =[]
    for synset in all_synsets:
        for each_name in synset.lemma_names():
            if each_name not in lemma_names:
                lemma_names.append(each_name)
            break
    return lemma_names

# mid level concepts
def get_general_concepts():
    #get all relevant sysnsets
    all_synsets = get_singles_doubles_synset()
    #get list of their hypernyms
    all_hyp = [each_hyp for syn in all_synsets for each_hyp in syn.hypernyms()]
    #extraxting general concepts
    lemma_names =[]
    for synset in all_hyp:
        for each_name in synset.lemma_names():
            if each_name not in lemma_names:
                lemma_names.append(each_name)
            break
    return lemma_names

#get common parents
def get_lcs():
    #get all relevant sysnsets
    all_synsets = get_singles_doubles_synset()
    list_of_all_lcs =[]
    for syn in all_synsets:
        for syn2 in all_synsets[all_synsets.index(syn)+1:]:
            lcs = syn.lowest_common_hypernyms(syn2)[0]
            if lcs not in list_of_all_lcs:
                list_of_all_lcs.append(lcs)
    return list_of_all_lcs

# get themes
def get_theme():
    # get common parent
    parent_sysnset = get_lcs()
    # filter out absolute top level and get lemma_names
    lemma_names =[]
    for synset in parent_sysnset:
        if synset.min_depth() != 0:
            #print(synset)
            for each_name in synset.lemma_names():
                if each_name not in lemma_names:
                    lemma_names.append(each_name)
                break
    return lemma_names
    



In [73]:
direct_concepts =get_direct_concept()
general_concepts = get_general_concepts()
themes = get_theme()

In [74]:
(len(direct_concepts),len(general_concepts),len(themes))
print("Common themes in the text")
for word in direct_concepts:
    print(word)
print('\n')

print("General concepts in the text")
for word in general_concepts:
    print(word)
print('\n')

print("Direct concepts in the text")
for word in themes:
    print(word)
print('\n')

Common themes in the text
trail
plan
use
bicycle
management
order
executive
information
connection
bacillus
park
valley
rule
overseer
office
notice
passenger_car
neptunium


General concepts in the text
path
idea
activity
wheeled_vehicle
social_control
command
administrator
message
instrumentality
eubacteria
tract
natural_depression
concept
supervisor
place_of_business
announcement
car
metallic_element


Direct concepts in the text
object
location
trail
physical_entity
psychological_feature
abstraction
plan
idea
act
use
whole
instrumentality
artifact
wheeled_vehicle
order
organism
leader
message
park




** Now we will try to create sentence summary from the text. This includes getting sentences from the text that would create a summary for the document **

In [76]:
def create_sentences(corpus):
    # First, tokenize the corpus into sentences
    sent_tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
    raw_sents = sent_tokenizer.tokenize(corpus)
    
    return raw_sents

def sentence_threshold(sent_list):
    

In [82]:
sentences = create_sentences(clean_suppl_info)